In [2]:
import uproot
import pandas as pd

In [7]:
# tree_tt = uproot.open("/eos/user/d/dwinterb/SWAN_projects/Masters_CP/MVAFILE_AllHiggs_tt.root")["ntuple"]
tree_tt = uproot.open("../MVAFILE_AllHiggs_tt_new.root")["ntuple"]
print('loaded root file')
variables = [
            "wt_cp_sm", "wt_cp_ps", "wt_cp_mm", "rand",
            "aco_angle_1", "aco_angle_5", "aco_angle_6", "aco_angle_7", 
            "mva_dm_1","mva_dm_2",
            "tau_decay_mode_1","tau_decay_mode_2",
            "ip_x_1", "ip_y_1", "ip_z_1", "ip_x_2", "ip_y_2", "ip_z_2", # ignore impact parameter for now
            "pi_E_1", "pi_px_1", "pi_py_1", "pi_pz_1", 
            "pi_E_2", "pi_px_2", "pi_py_2", "pi_pz_2", 
            "pi0_E_1", "pi0_px_1", "pi0_py_1", "pi0_pz_1",
            "pi0_E_2", "pi0_px_2", "pi0_py_2", "pi0_pz_2", 
            "y_1_1", "y_1_2",
            'met', 'metx', 'mety',
            'metcov00', 'metcov01', 'metcov10', 'metcov11',
#             'sv_x_1', 'sv_y_1', 'sv_z_1', 'sv_x_2', 'sv_y_2','sv_z_2'
        ]

variables += ["gen_nu_p_1", "gen_nu_phi_1", "gen_nu_eta_1", #leading neutrino, gen level
            "gen_nu_p_2", "gen_nu_phi_2", "gen_nu_eta_2" #subleading neutrino, gen level
             ]

df_tt = tree_tt.pandas.df(variables)
print('loaded df')

loaded root file
loaded df


In [4]:
tree_tt = uproot.open("./MVAFILE_AllHiggs_tt_new.root")["ntuple"]
dir(tree_tt)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__metaclass__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_addprovenance',
 '_attachstreamer',
 '_bases',
 '_branch_flags',
 '_branch_regex',
 '_branchlookup',
 '_classname',
 '_classversion',
 '_context',
 '_copycontext',
 '_dtype1',
 '_dtype2',
 '_fAliases',
 '_fAutoFlush',
 '_fAutoSave',
 '_fBranches',
 '_fClusterRangeEnd',
 '_fClusterSize',
 '_fDefaultEntryOffsetLen',
 '_fEntries',
 '_fEstimate',
 '_fFillColor',
 '_fFillStyle',
 '_fFlushedBytes',
 '_fFriends',
 '_fIndex',
 '_fIndexValues',
 '_fLeaves',
 '_fLineColor',
 '_fLineStyle',
 '_fLineWidth',
 '_fMarkerColor',
 '_fMarkerSize',
 '_fMarkerStyle

In [8]:
df_tt.to_pickle('./df_tt.pkl')

In [3]:
df_tt = pd.read_pickle('df_tt.pkl')

In [4]:
channel = 'rho_rho'

In [5]:
# reco level data
if channel == 'rho_rho':
    df_rho = df_tt[(df_tt['mva_dm_1']==1) & (df_tt['mva_dm_2']==1) & (df_tt["tau_decay_mode_1"] == 1) & (df_tt["tau_decay_mode_2"] == 1)]
    df = df_rho.drop(["mva_dm_1","mva_dm_2","tau_decay_mode_1","tau_decay_mode_2", "wt_cp_sm", "wt_cp_ps", "wt_cp_mm", "rand"], axis=1).reset_index(drop=True)
elif channel == 'rho_a1':
    df_rho_a1 = df_tt[(df_tt['mva_dm_1']==1) & (df_tt['mva_dm_2']==10) & (df_tt["tau_decay_mode_1"] == 1)]
    df = df_rho_a1.drop(["mva_dm_1","mva_dm_2","tau_decay_mode_1","tau_decay_mode_2", "wt_cp_sm", "wt_cp_ps", "wt_cp_mm", "rand"], axis=1).reset_index(drop=True)
elif channel == 'a1_a1':
    df_a1_a1 = df_tt[(df_tt['mva_dm_1']==10) & (df_tt['mva_dm_2']==10)]
    df = df_a1_a1.drop(["mva_dm_1","mva_dm_2","tau_decay_mode_1","tau_decay_mode_2", "wt_cp_sm", "wt_cp_ps", "wt_cp_mm", "rand"], axis=1).reset_index(drop=True)    
else:
    print('CHANNEL not understood!')

## testing data loader

In [12]:
from pylorentz import Momentum4
import numpy as np

In [7]:
df_rho = df_tt[(df_tt['mva_dm_1'] == 1) & (df_tt['mva_dm_2'] == 1) & (df_tt["tau_decay_mode_1"] == 1) & (df_tt["tau_decay_mode_2"] == 1)]

In [17]:
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

In [18]:
pi_1 = Momentum4(df['pi_E_1'], df["pi_px_1"], df["pi_py_1"], df["pi_pz_1"])
pi_2 = Momentum4(df['pi_E_2'], df["pi_px_2"], df["pi_py_2"], df["pi_pz_2"])
pi0_1 = Momentum4(df['pi0_E_1'], df["pi0_px_1"], df["pi0_py_1"], df["pi0_pz_1"])
pi0_2 = Momentum4(df['pi0_E_2'], df["pi0_px_2"], df["pi0_py_2"], df["pi0_pz_2"])
N = len(df['metx'])
met_x = Momentum4(df['metx'], np.zeros(N), np.zeros(N), np.zeros(N))
met_y = Momentum4(df['mety'], np.zeros(N), np.zeros(N), np.zeros(N))
met = Momentum4(df['met'], np.zeros(N), np.zeros(N), np.zeros(N))
rho_1 = pi_1 + pi0_1
rho_2 = pi_2 + pi0_2
# boost into rest frame of resonances
rest_frame = pi_1 + pi_2 + pi0_1 + pi0_2
boost = Momentum4(rest_frame[0], -rest_frame[1], -rest_frame[2], -rest_frame[3])
pi_1_boosted = pi_1.boost_particle(boost)
pi_2_boosted = pi_2.boost_particle(boost)
pi0_1_boosted = pi0_1.boost_particle(boost)
pi0_2_boosted = pi0_2.boost_particle(boost)
rho_1_boosted = pi_1_boosted + pi0_1_boosted
rho_2_boosted = pi_2_boosted + pi0_2_boosted
# boost MET - E_miss is already boosted into the hadronic rest frame
E_miss = met_x.boost_particle(boost)[0]
E_miss_x = met_y.boost_particle(boost)[0]
E_miss_y = met.boost_particle(boost)[0]
# rotations
pi_1_boosted_rot = []
pi_2_boosted_rot = []
pi0_1_boosted_rot = []
pi0_2_boosted_rot = []
rho_1_boosted_rot = []
rho_2_boosted_rot = []
for i in range(pi_1_boosted[:].shape[1]):
    rot_mat = rotation_matrix_from_vectors(rho_1_boosted[1:, i], [0, 0, 1])
    pi_1_boosted_rot.append(rot_mat.dot(pi_1_boosted[1:, i]))
    pi0_1_boosted_rot.append(rot_mat.dot(pi0_1_boosted[1:, i]))
    pi_2_boosted_rot.append(rot_mat.dot(pi_2_boosted[1:, i]))
    pi0_2_boosted_rot.append(rot_mat.dot(pi0_2_boosted[1:, i]))
    rho_1_boosted_rot.append(rot_mat.dot(rho_1_boosted[1:, i]))
    rho_2_boosted_rot.append(rot_mat.dot(rho_2_boosted[1:, i]))
    if i % 100000 == 0:
        print('finished getting rotated 4-vector', i)

finished getting rotated 4-vector 0
finished getting rotated 4-vector 100000
finished getting rotated 4-vector 200000
finished getting rotated 4-vector 300000
finished getting rotated 4-vector 400000
finished getting rotated 4-vector 500000
finished getting rotated 4-vector 600000
finished getting rotated 4-vector 700000
finished getting rotated 4-vector 800000
finished getting rotated 4-vector 900000


In [21]:
br_fourvec = np.array([pi_1_boosted_rot,pi_2_boosted_rot,pi0_1_boosted_rot,pi0_2_boosted_rot,rho_1_boosted_rot,rho_2_boosted_rot])

In [75]:
pi_1_boosted_rot = np.array(pi_1_boosted_rot)
pi_2_boosted_rot = np.array(pi_2_boosted_rot)
pi0_1_boosted_rot = np.array(pi0_1_boosted_rot)
pi0_2_boosted_rot = np.array(pi0_2_boosted_rot)
rho_1_boosted_rot = np.array(rho_1_boosted_rot)
rho_2_boosted_rot = np.array(rho_2_boosted_rot)

In [76]:
df_inputs_data = {
    'pi_E_1_br': pi_1_boosted[0],
    'pi_px_1_br': pi_1_boosted_rot[:,0],
    'pi_py_1_br': pi_1_boosted_rot[:,1],
    'pi_pz_1_br': pi_1_boosted_rot[:,2],
    'pi_E_2_br': pi_2_boosted[0],
    'pi_px_2_br': pi_2_boosted_rot[:,0],
    'pi_py_2_br': pi_2_boosted_rot[:,1],
    'pi_pz_2_br': pi_2_boosted_rot[:,2],
    'pi0_E_1_br': pi0_1_boosted[0],
    'pi0_px_1_br': pi0_1_boosted_rot[:,0],
    'pi0_py_1_br': pi0_1_boosted_rot[:,1],
    'pi0_pz_1_br': pi0_1_boosted_rot[:,2],
    'pi0_E_2_br': pi0_2_boosted[0],
    'pi0_px_2_br': pi0_2_boosted_rot[:,0],
    'pi0_py_2_br': pi0_2_boosted_rot[:,1],
    'pi0_pz_2_br': pi0_2_boosted_rot[:,2],
    'rho_E_1_br': rho_1_boosted[0],
    'rho_px_1_br': rho_1_boosted_rot[:,0],
    'rho_py_1_br': rho_1_boosted_rot[:,1],
    'rho_pz_1_br': rho_1_boosted_rot[:,2],
    'rho_E_2_br': rho_2_boosted[0],
    'rho_px_2_br': rho_2_boosted_rot[:,0],
    'rho_py_2_br': rho_2_boosted_rot[:,1],
    'rho_pz_2_br': rho_2_boosted_rot[:,2],
    'aco_angle_1': df['aco_angle_1'],
    'y_1_1': df['y_1_1'],
    'y_1_2': df['y_1_2'],
#     'w_a': df['wt_cp_sm'],
#     'w_b': df['wt_cp_ps'],
    'm_1': rho_1.m,
    'm_2': rho_2.m,
}

In [77]:
df_inputs = pd.DataFrame(df_inputs_data)

In [78]:
df_inputs.head()

,pi_E_1_br,pi_px_1_br,pi_py_1_br,pi_pz_1_br,pi_E_2_br,pi_px_2_br,pi_py_2_br,pi_pz_2_br,pi0_E_1_br,pi0_px_1_br,...,rho_pz_1_br,rho_E_2_br,rho_px_2_br,rho_py_2_br,rho_pz_2_br,aco_angle_1,y_1_1,y_1_2,m_1,m_2
0,38.866249,0.429429,-0.133119,38.863398,40.456445,-0.509745,-0.094749,-40.452882,8.569687,-0.429429,...,47.420221,47.444351,2.442491e-14,5.329071e-14,-47.420221,2.657029,-0.637964,-0.708372,1.220952,1.512969
1,17.175150,0.213861,0.469040,17.166846,9.529103,0.210109,-0.089910,-9.525340,29.088943,-0.213861,...,46.250908,46.255629,5.329071e-15,-1.154632e-14,-46.250908,1.547246,0.254573,0.594557,1.104476,0.660891
2,8.526993,0.209425,0.197722,8.520985,10.630995,-0.242771,-0.036193,-10.627246,37.577339,-0.209425,...,46.096978,46.101834,2.131628e-14,7.105427e-15,-46.096978,3.750274,0.638590,0.529623,0.823420,0.669092
3,41.836730,-0.050458,-0.144726,41.836217,2.599469,0.099976,0.004286,-2.593793,11.395560,0.050458,...,53.229947,53.235902,4.352074e-14,-3.197442e-14,-53.229947,1.192612,-0.572846,0.899114,0.499415,0.796268
4,23.455746,0.115471,-0.337611,23.452617,25.232731,-0.068646,0.327903,-25.230121,21.066138,-0.115471,...,44.515301,44.521292,-1.143530e-14,-9.547918e-15,-44.515301,3.264792,-0.054617,-0.132105,0.765599,0.730365


In [44]:
np.array(df_inputs.pi_1_br.tolist()).shape

(998268, 3)

In [73]:
df_inputs[['pi_E_1_br','pi_px_1_br','pi_py_1_br','pi_pz_1_br']] = pd.DataFrame(np.array(pi_1_boosted_rot))

ValueError: Columns must be same length as key